# Phase 2: Mining Refactoring Candidates

**Objective:** To traverse the git history of the identified high-quality repositories and extract commit data related to refactoring activities.

**Methodology:**
1.  **Robust Loading:** We automatically detect and load the most recent target list generated in Phase 1 (e.g., `vr_repo_targets_20251207.csv`).
2.  **Iterative Analysis:** To prevent Google Colab disk saturation, we clone one repository at a time, analyze it, and immediately delete it.
3.  **Maturity Validation:** We apply the final filter (>100 commits) locally using `git` commands, which is instant and avoids API limits.
4.  **Keyword Detection:** We scan commit messages for standard refactoring keywords (e.g., "refactor", "restructure", "clean up") as established in MSR literature.
5.  **Artifact Categorization:** We detect which types of files were modified (Scripts vs. Assets) to enable categorization of "Code Refactors" vs. "Scene Refactors".

In [ ]:
# @title 1. Install PyDriller & Setup Environment
!pip install pydriller tqdm -q

import pandas as pd
import os
import glob
import shutil
import subprocess
from pydriller import Repository
from google.colab import drive
from datetime import datetime
from tqdm.notebook import tqdm

# 1. Mount Drive
if not os.path.exists('/content/drive'):
    print("Mounting Google Drive...")
    drive.mount('/content/drive')

# 2. Robust File Loader
def get_latest_target_file(folder_path):
    """
    Finds the CSV file with the latest timestamp in the filename.
    Pattern: vr_repo_targets_YYYYMMDD.csv
    """
    # Search for files matching the pattern
    search_pattern = os.path.join(folder_path, "vr_repo_targets_*.csv")
    files = glob.glob(search_pattern)

    if not files:
        raise FileNotFoundError(f"No target files found in {folder_path}. Did you run Phase 1?")

    # Sort files by name (which effectively sorts by timestamp YYYYMMDD)
    latest_file = sorted(files)[-1]
    print(f"✅ Found latest target list: {os.path.basename(latest_file)}")
    return latest_file

# Load the Data
base_folder = "/content/drive/My Drive/VR_Refactoring_Study"
target_file = get_latest_target_file(base_folder)
df_targets = pd.read_csv(target_file)

print(f"Loaded {len(df_targets)} repositories for mining.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 2.9 MB/s eta 0:00:00
Mounting Google Drive...
Mounted at /content/drive
✅ Found latest target list: vr_repo_targets_20260201.csv
Loaded 224 repositories for mining.


In [ ]:
# @title 2. The Mining Engine (Cloning & Filtering)

# --- CONFIGURATION ---
KEYWORDS = [
    "refactor", "cleanup", "clean up", "restructure", "organize",
    "simplify", "extract", "move", "rename", "reorganize"
]
TEMP_REPO_DIR = "/content/temp_repo"

def count_commits_local(repo_path):
    """Runs git rev-list to get an instant commit count."""
    try:
        result = subprocess.run(
            ["git", "rev-list", "--count", "HEAD"],
            cwd=repo_path, capture_output=True, text=True, check=True
        )
        return int(result.stdout.strip())
    except Exception as e:
        print(f"      [!] Error counting commits: {e}")
        return 0

def analyze_repo(repo_url, repo_name, project_type):
    """Clones, filters, mines, and cleans up a single repo."""
    candidates = []
    print(f"\n🔹 Processing: {repo_name} ({project_type})")

    # 1. Clean up any previous temp runs
    if os.path.exists(TEMP_REPO_DIR):
        shutil.rmtree(TEMP_REPO_DIR)

    try:
        # 2. Clone locally
        print(f"   [1/4] Cloning from {repo_url}...")
        subprocess.run(["git", "clone", repo_url, TEMP_REPO_DIR],
                       check=True, capture_output=True)

        # 3. Filter: Maturity Check (> 100 commits)
        total_commits = count_commits_local(TEMP_REPO_DIR)
        print(f"   [2/4] Maturity Check: Found {total_commits} total commits.")

        # if total_commits < 100:
        #     print(f"   ❌ Skipped: Too few commits (< 100).")
        #     return [], "Too few commits"

        # 4. Traverse Commits using PyDriller
        print(f"   [3/4] Mining git history for keywords...")
        repo_miner = Repository(TEMP_REPO_DIR)

        hit_count = 0
        for commit in repo_miner.traverse_commits():
            msg_lower = commit.msg.lower()

            # Check for keywords
            if any(keyword in msg_lower for keyword in KEYWORDS):
                hit_count += 1

                # 5. Extract File Metadata
                files_changed = []
                extensions = set()

                for mod in commit.modified_files:
                    files_changed.append(mod.filename)
                    # Get extension (e.g., .cs, .unity)
                    ext = os.path.splitext(mod.filename)[1].lower()
                    extensions.add(ext)

                # 6. Infer Refactor Type based on extensions
                refactor_type = "Other"
                if ".cs" in extensions:
                    refactor_type = "Code Refactor"
                elif ".unity" in extensions or ".prefab" in extensions:
                    refactor_type = "Scene/Prefab Refactor"
                elif ".mat" in extensions or ".shader" in extensions or ".meta" in extensions:
                    refactor_type = "Asset Refactor"

                candidates.append({
                    "project_name": repo_name,
                    "project_type": project_type,
                    "commit_hash": commit.hash,
                    "date": commit.committer_date,
                    "author": commit.author.name,
                    "message": commit.msg,
                    "files_changed": "; ".join(files_changed), # flattening list for CSV
                    "refactor_category": refactor_type,
                    "repo_url": repo_url
                })

        print(f"   [4/4] Complete. Found {hit_count} candidates.")

        # Cleanup
        shutil.rmtree(TEMP_REPO_DIR)
        return candidates, "Success"

    except Exception as e:
        print(f"   [!] CRITICAL ERROR: {str(e)}")
        if os.path.exists(TEMP_REPO_DIR):
            shutil.rmtree(TEMP_REPO_DIR)
        return [], f"Error: {str(e)}"

In [ ]:
# @title 3. Execution Loop
# Iterate through the target list and mine data

all_refactors = []
skipped_repos = []

print(f"🚀 Starting Mining Operation on {len(df_targets)} repositories...")
print("This may take a while. Progress bar below.\n")

# TQDM gives us a nice progress bar
for index, row in tqdm(df_targets.iterrows(), total=df_targets.shape[0]):
    repo_name = row['name']
    repo_url = row['url']
    p_type = row['project_type']

    # Run the analysis
    results, status = analyze_repo(repo_url, repo_name, p_type)

    if status == "Success":
        all_refactors.extend(results)
    else:
        skipped_repos.append({"name": repo_name, "reason": status})

# --- SAVING RESULTS ---
timestamp = datetime.now().strftime("%Y%m%d")
output_filename = f"refactoring_candidates_{timestamp}.csv"
output_path = os.path.join(base_folder, output_filename)

# Convert to DataFrame
df_results = pd.DataFrame(all_refactors)

# Save
if not df_results.empty:
    df_results.to_csv(output_path, index=False)
    print("\n" + "="*50)
    print(f"✅ Mining Complete!")
    print(f"📊 Total Candidates Found: {len(df_results)}")
    print(f"📂 Saved to: {output_path}")
    print("="*50)

    # Show preview
    print("\nPreview of Findings:")
    print(df_results[['project_name', 'refactor_category', 'message']].head())
else:
    print("⚠️ No refactoring candidates found. Check your keywords or repo list.")

# Optional: Save skipped repos log
if skipped_repos:
    log_path = os.path.join(base_folder, f"skipped_repos_{timestamp}.csv")
    pd.DataFrame(skipped_repos).to_csv(log_path, index=False)
    print(f"ℹ️  Skipped repository log saved to: {log_path}")

🚀 Starting Mining Operation on 224 repositories...
This may take a while. Progress bar below.



  0%|          | 0/224 [00:00<?, ?it/s]


🔹 Processing: VRTK (Organizational)
   [1/4] Cloning from https://github.com/ExtendRealityLtd/VRTK.git...
   [2/4] Maturity Check: Found 2473 total commits.
   [3/4] Mining git history for keywords...
   [4/4] Complete. Found 468 candidates.

🔹 Processing: InputSystem (Organizational)
   [1/4] Cloning from https://github.com/Unity-Technologies/InputSystem.git...
   [2/4] Maturity Check: Found 2612 total commits.
   [3/4] Mining git history for keywords...
   [4/4] Complete. Found 362 candidates.

🔹 Processing: open-brush (Organizational)
   [1/4] Cloning from https://github.com/icosa-foundation/open-brush.git...
   [2/4] Maturity Check: Found 851 total commits.
   [3/4] Mining git history for keywords...
   [4/4] Complete. Found 177 candidates.

🔹 Processing: EditorXR (Organizational)
   [1/4] Cloning from https://github.com/Unity-Technologies/EditorXR.git...
   [2/4] Maturity Check: Found 6176 total commits.
   [3/4] Mining git history for keywords...
   [4/4] Complete. Found 1506 ca